In [0]:
## Prepare data
from pyspark.sql import functions as F

# Load silver events
events = spark.table("ecommerce.silver.events")

# Simple feature set
ml_data = (
    events
    .filter(F.col("price").isNotNull())
    .withColumn("hour", F.hour("event_time"))
    .withColumn("day_of_week", F.dayofweek("event_time"))
    .select("price", "hour", "day_of_week")
)


In [0]:
## Vectorize features
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["hour", "day_of_week"],
    outputCol="features"
)

final_data = assembler.transform(ml_data).select("features", "price")

## Train-test split
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

## Train Linear Regression model
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="price")

model = lr.fit(train_data)


In [0]:
dbutils.fs.mkdirs("dbfs:/Volumes/workspace/default/kaggle_volume/mlflow_tmp")


True

In [0]:
import os
os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/kaggle_volume/mlflow_tmp"


In [0]:
## Log parameters, metrics, model
import os
import mlflow
import mlflow.spark

# Required for Unity Catalog
os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/kaggle_volume/mlflow_tmp"

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("features", "hour, day_of_week")

    # Predictions
    predictions = model.transform(test_data)

    from pyspark.ml.evaluation import RegressionEvaluator
    evaluator = RegressionEvaluator(
        labelCol="price",
        predictionCol="prediction",
        metricName="rmse" 
    )

    rmse = evaluator.evaluate(predictions)

    # Log metric
    mlflow.log_metric("rmse", rmse)
    
    # Log model
    mlflow.spark.log_model(model, "linear_regression_model")

    print("RMSE:", rmse)


2026/01/20 07:07:21 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/20 07:07:26 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-831b0ef4-205c-49ee-b27d-13/tmp7_fw6eg7/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/20 07:07:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RMSE: 357.99881212509115


In [0]:
## Compare Runs

## Simple Features
with mlflow.start_run():
    mlflow.log_param("features", "hour, day_of_week")
    mlflow.log_param("model", "LinearRegression")
    mlflow.log_metric("rmse", rmse)

## Add more features
with mlflow.start_run():
    mlflow.log_param("features", "hour, day_of_week, is_weekend")
    mlflow.log_param("model", "LinearRegression")
    mlflow.log_metric("rmse", rmse)

